references: https://www.kaggle.com/code/fengliplatform/customer-sentiment-analysis

In [1]:
# generate ticket priority
# based on overall sentiment score (VADER), polarity, and topic frequency

# complaint subjectivity (0-1), where one is extremely subjective, 
# the complaint polarity (-1 to 1)
# topic frequency is topic frequency/total

# priority = subjectivity - polarity + topic_frequency

# then abc ranking (20-30-50)

In [2]:
import pandas as pd
import json
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

# import nltk
# nltk.download('vader_lexicon')

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
def get_processed_df(csv_path):
    df = pd.read_csv(csv_path)
    return df

In [4]:
df = get_processed_df('process_csv_stage_2.csv')
df

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp
0,1,good morning name appreciate could help put st...,0,5,0,34
1,2,upgraded card tell agent upgrade anniversary d...,3,3,0,2
2,10,chase card report however fraudulent applicati...,6,2,0,16
3,11,try book ticket come across offer apply toward...,3,7,0,24
4,14,grand son give check deposit chase account fun...,4,4,0,0
...,...,...,...,...,...,...
21067,78303,chase card customer well decade offer multiple...,3,2,0,2
21068,78309,wednesday call chas visa credit card provider ...,8,5,0,39
21069,78310,familiar pay understand great risk provide con...,2,1,0,9
21070,78311,flawless credit chase credit card chase freedo...,7,7,0,6


In [5]:
# subjectivity
def get_subjectivity(text):
   return TextBlob(text).sentiment.subjectivity

In [6]:
# polarity
sentiment_analyzer = SentimentIntensityAnalyzer()

def get_compound_polarity(text: str) -> float:
    return sentiment_analyzer.polarity_scores(text)['compound']

In [7]:
df['subjectivity'] = df['preprocessed_text'].apply(get_subjectivity)
df['compound_polarity'] = df['preprocessed_text'].apply(get_compound_polarity)

In [8]:
# topic frequency
# we choose topic model nmf
# topic frequency = topic_count / total_count
chosen_topic_model = 'topic_lda'
topic_frequencies = pd.DataFrame(df.value_counts(chosen_topic_model)/df.shape[0]).reset_index().rename(columns={'count':'topic_frequency'})
df = pd.merge(left = df, right=topic_frequencies,on=chosen_topic_model,how='inner').sort_values(by=['old_index'],ascending=True).reset_index(drop=True)

In [9]:
df

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency
0,1,good morning name appreciate could help put st...,0,5,0,34,0.445238,0.8402,0.197134
1,2,upgraded card tell agent upgrade anniversary d...,3,3,0,2,0.900000,-0.5812,0.095292
2,10,chase card report however fraudulent applicati...,6,2,0,16,0.000000,-0.1446,0.115556
3,11,try book ticket come across offer apply toward...,3,7,0,24,0.471429,0.9058,0.095292
4,14,grand son give check deposit chase account fun...,4,4,0,0,0.753333,0.9215,0.151481
...,...,...,...,...,...,...,...,...,...
21067,78303,chase card customer well decade offer multiple...,3,2,0,2,0.291288,0.6369,0.095292
21068,78309,wednesday call chas visa credit card provider ...,8,5,0,39,0.324318,-0.9377,0.095008
21069,78310,familiar pay understand great risk provide con...,2,1,0,9,0.468801,0.0641,0.045416
21070,78311,flawless credit chase credit card chase freedo...,7,7,0,6,0.475247,0.9716,0.118641


In [10]:
df.sort_values(by=['subjectivity'],ascending=False)

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency
14154,47890,give authorization help bank dispute behalf un...,5,3,0,18,1.0,-0.6369,0.066297
7466,21889,chase bank mortgage account bing hold chase re...,2,6,0,38,1.0,-0.8689,0.045416
1105,3563,never own account,6,3,5,82,1.0,0.0000,0.115556
20614,75849,deal seller transfer fund though soon send mon...,0,4,0,9,1.0,-0.7367,0.197134
4436,12294,charge airport lounge service wonder included ...,8,1,0,60,1.0,0.8074,0.095008
...,...,...,...,...,...,...,...,...,...
4929,14986,allow vender charge account item order cause l...,8,7,0,1,0.0,0.2769,0.095008
20266,75037,buy fully unlock silver sell package deliver p...,8,5,0,20,0.0,-0.4215,0.095008
15516,54741,want add motion information file federal bankr...,2,5,0,35,0.0,0.4767,0.045416
11667,32897,account result fraud,6,3,5,21,0.0,-0.5859,0.115556


In [11]:
df.sort_values(by=['compound_polarity'],ascending=True)

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency
17655,64994,fraud ing scam use document fraud ing indiana ...,5,5,0,45,0.483770,-1.0000,0.066297
5295,15832,fraud ing scam use indiana document fraud ing ...,5,5,0,45,0.403480,-0.9999,0.066297
5289,15814,den ged credit card throw card company card co...,5,6,0,66,0.025000,-0.9996,0.066297
5580,17342,last vacation experience bank fraud begin shor...,5,1,0,48,0.342283,-0.9993,0.066297
9467,28689,open credit card chase approve amazon credit c...,5,2,0,7,0.448457,-0.9993,0.066297
...,...,...,...,...,...,...,...,...,...
4741,14476,forward message date wed subject fwd follow po...,0,5,0,46,0.504554,0.9999,0.197134
14251,48112,urgent president commissary majority owner pro...,2,5,0,64,0.360492,0.9999,0.045416
5880,18007,reason apply receive mile sign bonus chase den...,3,6,0,2,0.436166,0.9999,0.095292
4662,14281,apply approve brand card chase accord term con...,3,2,0,2,0.448617,0.9999,0.095292


In [12]:
df.sort_values(by=['topic_frequency'],ascending=False).groupby('topic_frequency').head(2)

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency
0,1,good morning name appreciate could help put st...,0,5,0,34,0.445238,0.8402,0.197134
4898,14898,hello amazon leg loose jumpsuit romper due ama...,0,5,0,20,0.364760,0.3173,0.197134
19380,71963,schedule close house cash deal money wire titl...,4,3,0,18,0.299242,0.6540,0.151481
12225,34173,chase banking copy bank statement loan know no...,4,3,0,-1,0.000000,-0.1531,0.151481
17431,62118,online bank usual chase credit card southwest ...,7,1,0,26,0.431723,-0.8176,0.118641
20755,76603,make payment phone payment process correctly c...,7,7,0,30,0.525000,0.2023,0.118641
6675,19789,report jpmorgan chase one account fraudulently...,6,8,0,43,0.400655,-0.9042,0.115556
11783,33152,discover identity steal upon review credit rep...,6,5,0,90,0.309375,-0.9531,0.115556
5133,15519,recently purchase home new since work hard bui...,1,6,0,25,0.410768,-0.7906,0.115177
14332,48842,chase say would water people behind mort age n...,1,0,4,4,0.700000,0.2235,0.115177


In [13]:
df['priority_score'] = df['subjectivity'] - df['compound_polarity'] + df['topic_frequency']
df

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency,priority_score
0,1,good morning name appreciate could help put st...,0,5,0,34,0.445238,0.8402,0.197134,-0.197828
1,2,upgraded card tell agent upgrade anniversary d...,3,3,0,2,0.900000,-0.5812,0.095292,1.576492
2,10,chase card report however fraudulent applicati...,6,2,0,16,0.000000,-0.1446,0.115556,0.260156
3,11,try book ticket come across offer apply toward...,3,7,0,24,0.471429,0.9058,0.095292,-0.339079
4,14,grand son give check deposit chase account fun...,4,4,0,0,0.753333,0.9215,0.151481,-0.016686
...,...,...,...,...,...,...,...,...,...,...
21067,78303,chase card customer well decade offer multiple...,3,2,0,2,0.291288,0.6369,0.095292,-0.250320
21068,78309,wednesday call chas visa credit card provider ...,8,5,0,39,0.324318,-0.9377,0.095008,1.357026
21069,78310,familiar pay understand great risk provide con...,2,1,0,9,0.468801,0.0641,0.045416,0.450117
21070,78311,flawless credit chase credit card chase freedo...,7,7,0,6,0.475247,0.9716,0.118641,-0.377712


In [14]:
sort_by_priority_score = df.sort_values(by=['priority_score'],ascending=False)
sort_by_priority_score

,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency,priority_score
3810,10981,make shipment chase bank say would arrive busi...,0,0,0,0,1.000000,-0.8625,0.197134,2.059634
3760,10863,try contact dispute dept morgan chase bank pdt...,0,1,0,-1,0.850000,-0.9501,0.197134,1.997234
5840,17904,dad mom account credit card chase die mom file...,0,3,0,50,0.833333,-0.9485,0.197134,1.978967
20413,75383,still fight hear headset voip system refuse ge...,0,0,0,27,0.820000,-0.9538,0.197134,1.970934
3348,8943,call magazine debt collector call telling orde...,0,0,0,15,1.000000,-0.7717,0.197134,1.968834
...,...,...,...,...,...,...,...,...,...,...
20495,75536,turn leased vehicle per lease agreement expire...,7,5,0,41,0.000000,0.9628,0.118641,-0.844159
10408,30025,file chapter bankruptcy dish arge bankruptcy i...,2,6,0,35,0.000000,0.9042,0.045416,-0.858784
367,923,chase credit card close without tell chase ref...,3,2,0,2,0.000000,0.9595,0.095292,-0.864208
16597,61254,apply approve southwest credit card use card c...,3,2,0,2,0.000000,0.9657,0.095292,-0.870408


In [15]:
def abc_ranking(priority_rank_score):
    # priority_rank_score value is ranged from 0 to 1
    if priority_rank_score <=0.2:
        return '1' # high priority
    elif priority_rank_score > 0.2 and priority_rank_score <=0.5:
        return '2' # medium priority
    else: # priority_rank_score > 0.5 and priority_rank_score <=1
        return '3' # low priority

In [16]:
calculate_priority_df = sort_by_priority_score.reset_index(names=['current_index']).reset_index(names=['sort_index'])
calculate_priority_df

,sort_index,current_index,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency,priority_score
0,0,3810,10981,make shipment chase bank say would arrive busi...,0,0,0,0,1.000000,-0.8625,0.197134,2.059634
1,1,3760,10863,try contact dispute dept morgan chase bank pdt...,0,1,0,-1,0.850000,-0.9501,0.197134,1.997234
2,2,5840,17904,dad mom account credit card chase die mom file...,0,3,0,50,0.833333,-0.9485,0.197134,1.978967
3,3,20413,75383,still fight hear headset voip system refuse ge...,0,0,0,27,0.820000,-0.9538,0.197134,1.970934
4,4,3348,8943,call magazine debt collector call telling orde...,0,0,0,15,1.000000,-0.7717,0.197134,1.968834
...,...,...,...,...,...,...,...,...,...,...,...,...
21067,21067,20495,75536,turn leased vehicle per lease agreement expire...,7,5,0,41,0.000000,0.9628,0.118641,-0.844159
21068,21068,10408,30025,file chapter bankruptcy dish arge bankruptcy i...,2,6,0,35,0.000000,0.9042,0.045416,-0.858784
21069,21069,367,923,chase credit card close without tell chase ref...,3,2,0,2,0.000000,0.9595,0.095292,-0.864208
21070,21070,16597,61254,apply approve southwest credit card use card c...,3,2,0,2,0.000000,0.9657,0.095292,-0.870408


In [17]:
# abc ranking (20-30-50)
calculate_priority_df['priority_rank_score'] = (calculate_priority_df['sort_index']+1)/df.shape[0]
calculate_priority_df['priority'] = calculate_priority_df['priority_rank_score'].apply(abc_ranking)

In [18]:
calculate_priority_df

,sort_index,current_index,old_index,preprocessed_text,topic_lda,topic_nmf,topic_t2v,topic_btp,subjectivity,compound_polarity,topic_frequency,priority_score,priority_rank_score,priority
0,0,3810,10981,make shipment chase bank say would arrive busi...,0,0,0,0,1.000000,-0.8625,0.197134,2.059634,0.000047,1
1,1,3760,10863,try contact dispute dept morgan chase bank pdt...,0,1,0,-1,0.850000,-0.9501,0.197134,1.997234,0.000095,1
2,2,5840,17904,dad mom account credit card chase die mom file...,0,3,0,50,0.833333,-0.9485,0.197134,1.978967,0.000142,1
3,3,20413,75383,still fight hear headset voip system refuse ge...,0,0,0,27,0.820000,-0.9538,0.197134,1.970934,0.000190,1
4,4,3348,8943,call magazine debt collector call telling orde...,0,0,0,15,1.000000,-0.7717,0.197134,1.968834,0.000237,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21067,21067,20495,75536,turn leased vehicle per lease agreement expire...,7,5,0,41,0.000000,0.9628,0.118641,-0.844159,0.999810,3
21068,21068,10408,30025,file chapter bankruptcy dish arge bankruptcy i...,2,6,0,35,0.000000,0.9042,0.045416,-0.858784,0.999858,3
21069,21069,367,923,chase credit card close without tell chase ref...,3,2,0,2,0.000000,0.9595,0.095292,-0.864208,0.999905,3
21070,21070,16597,61254,apply approve southwest credit card use card c...,3,2,0,2,0.000000,0.9657,0.095292,-0.870408,0.999953,3


In [19]:
calculate_priority_df.value_counts('priority')

priority
3    10536
2     6322
1     4214
Name: count, dtype: int64

In [20]:
calculate_priority_df.value_counts('priority')/df.shape[0]*100

priority
3    50.000000
2    30.001898
1    19.998102
Name: count, dtype: float64

In [21]:
calculate_priority_df.set_index('current_index',inplace=True)
calculate_priority_df.drop(['sort_index','subjectivity','compound_polarity','topic_frequency','priority_score','priority_rank_score'],axis=1,inplace=True)
abandoned_topic_models = ['topic_nmf','topic_t2v','topic_btp']
calculate_priority_df.drop(abandoned_topic_models,axis=1,inplace=True)

In [22]:
calculate_priority_df.index.name = None
calculate_priority_df.sort_index(axis=0,inplace=True)

In [24]:
calculate_priority_df.to_csv('process_csv_stage_3.csv',index=False)